In [1]:
!wget https://raw.githubusercontent.com/happylolonly/events-free-ml/master/tags/data/data.csv

--2018-12-18 13:45:55--  https://raw.githubusercontent.com/happylolonly/events-free-ml/master/tags/data/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.248.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.248.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 417624 (408K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>] 407.84K  --.-KB/s    in 0.01s   

2018-12-18 13:45:55 (29.9 MB/s) - ‘data.csv’ saved [417624/417624]



In [89]:
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.bin.gz

--2018-12-18 14:13:18--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.bin.gz
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 54.231.237.33
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|54.231.237.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4496459151 (4.2G) [application/octet-stream]
Saving to: ‘cc.ru.300.bin.gz’

cc.ru.300.bin.gz    100%[===================>]   4.19G  18.8MB/s    in 19m 56s 

2018-12-18 14:33:15 (3.59 MB/s) - ‘cc.ru.300.bin.gz’ saved [4496459151/4496459151]



In [90]:
!wget https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz

--2018-12-18 14:33:15--  https://s3-us-west-1.amazonaws.com/fasttext-vectors/word-vectors-v2/cc.ru.300.vec.gz
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 54.231.237.29
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|54.231.237.29|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1306357571 (1.2G) [binary/octet-stream]
Saving to: ‘cc.ru.300.vec.gz’

cc.ru.300.vec.gz    100%[===================>]   1.22G  25.1MB/s    in 50s     

2018-12-18 14:34:05 (24.7 MB/s) - ‘cc.ru.300.vec.gz’ saved [1306357571/1306357571]



In [91]:
!gunzip cc.ru.300.bin.gz
!gunzip cc.ru.300.vec.gz

In [1]:
import pandas as pd

/home/afridman/.virtualenvs/jupyter/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/afridman/.virtualenvs/jupyter/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('data.csv')
df.head()

df['tags'] = df['tags'].map(eval) 
tags_white_list = {'митап', 'музыка', 'тренировка', 'дети', 'вечеринка'}
df['tags'] = df['tags'].map(lambda x: set(x).intersection(tags_white_list))
df = df[df.tags.map(len) > 0]

df.head()

,text,title,tags
1,🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 \n \nВ э...,🔥🔥🔥PoisonBox дарит возможности!!! 🔥🔥🔥 \n \nВ ...,{тренировка}
2,БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! \n...,БЕСПЛАТНАЯ открытая тренировка по CROSSFIT! \...,{тренировка}
4,Внимание!!!📌\n\nОткрытое занятие по TRX и SUPE...,Внимание!!!📌\n\nОткрытое занятие по TRX и SUP...,{тренировка}
5,"📌 Открытое занятие по TRX и SUPER 8, а так же ...","📌 Открытое занятие по TRX и SUPER 8, а так же...",{тренировка}
8,\n &#x415;&#x436;&#x435...,Cальса-вечеринка «Винтажные Грувы»,{вечеринка}


In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer('russian')

import json
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def standardize_text(df, text_field):
    df[text_field] = df[text_field].apply(strip_tags)
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"[^а-яА-Я0-9a-zA-Z]", " ")
    df[text_field] = df[text_field].str.replace(r"\s+", " ")
    df[text_field] = df[text_field].str.lower()
    return df

/home/afridman/.virtualenvs/jupyter/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/afridman/.virtualenvs/jupyter/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/afridman/.virtualenvs/jupyter/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/afridman/.virtualenvs/jupyter/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


[nltk_data] Downloading package wordnet to /home/afridman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = standardize_text(df, 'text')
df = standardize_text(df, 'title')

df.head()

,text,title,tags
1,poisonbox дарит возможности n nв эту субботу ...,poisonbox дарит возможности n nв эту субботу ...,{тренировка}
2,бесплатная открытая тренировка по crossfit n n...,бесплатная открытая тренировка по crossfit n ...,{тренировка}
4,внимание n nоткрытое занятие по trx и super 8 ...,внимание n nоткрытое занятие по trx и super 8...,{тренировка}
5,открытое занятие по trx и super 8 а так же на...,открытое занятие по trx и super 8 а так же на...,{тренировка}
8,n еженедельная сальса вечеринка винтажные гру...,cальса вечеринка винтажные грувы,{вечеринка}


In [5]:
from collections import Counter

counter = Counter()
for tags in df['tags'].values:
    for t in tags:
        counter[t] += 1

In [6]:
counter

Counter({'вечеринка': 6,
         'дети': 7,
         'митап': 15,
         'музыка': 12,
         'тренировка': 8})

In [7]:
rows = []

for row in df.to_dict(orient='record'):
    for tag in row['tags']:
        row_copy = row.copy()
        row_copy['tag'] = tag
        
        rows.append(row_copy)
        
df = pd.DataFrame(rows)

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.4, stratify=df['tag'].values)

In [9]:
import fastText

FIELD = 'text'

In [10]:
def save(df, path):
    with open(path, 'w+') as f:
        for _, row in df.iterrows():
            f.write('__label__{} {}\n'.format(row['tag'], row[FIELD]))

In [11]:
save(train, '/tmp/train.data')

In [ ]:
model = fastText.train_supervised('/tmp/train.data',
                                  pretrainedVectors='cc.ru.300.vec',
                                  dim=300,
                                  thread=15,
                                  epoch=50,
                                  verbose=8)

In [ ]:
labels, probs = model.predict(test[FIELD].tolist())
labels = [ll[0].replace('__label__', '') for ll in labels]

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test['tag'].values, labels))